In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
dataset = load_dataset("Abirate/english_quotes", split="train[:1000]")

tokenizer = AutoTokenizer.from_pretrained(model_id)
def tokenize_fn(example):
    return tokenizer("### Instruction:\n" + example["quote"] + "\n### Response:\n", truncation=True)
tokenized_ds = dataset.map(tokenize_fn, batched=True)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)

args = TrainingArguments(output_dir="./results", per_device_train_batch_size=1, gradient_accumulation_steps=4, logging_steps=10,
                         learning_rate=2e-4, num_train_epochs=1, fp16=True, save_total_limit=1)
trainer = Trainer(model=model, args=args, train_dataset=tokenized_ds)
trainer.train()